In [1]:
!pip install diffusers transformers safetensors accelerate
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 27.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylin

In [2]:
!pip install torch transformers diffusers gtts moviepy pysrt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=87d2e9ef5c910f10d7fe33933355fdcc0fa7288fb11e21b5fd8883712670d0f8
  Stored in directory: /root/.cache/pip/wheels/30/7f/e8/55de9a9b07302d9e7fe47c27910e3bea0c48536153e74bd7e6
Successfully built pysrt


In [3]:
!pip install torch transformers sentencepiece


In [4]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.4 MB/s eta 0:00:00


In [5]:
!pip install moviepy
!pip install pysrt
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 90.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import os
import random
import sys
import torch
import spacy
import pysrt
from moviepy.editor import concatenate_videoclips, ImageClip, AudioFileClip

# Function to generate story using GPT-2


# Function to extract keywords from the story for image prompts
def extract_image_prompts(story, num_prompts=6):
    nlp = spacy.load('en_core_web_sm')
    # Custom list of uninformative words
    uninformative_words = ['can', 'to', 'which', 'you', 'your', 'that','their','they']
    # Split the story into individual sentences
    doc = nlp(story)
    sentences = [sent.text.strip() for sent in doc.sents]
    # Find the main subject or noun phrase in each sentence
    main_subjects = []
    for sentence in sentences:
        doc = nlp(sentence.lower())
        for chunk in doc.noun_chunks:
            if chunk.root.dep_ == 'nsubj' and chunk.root.head.text.lower() != 'that':
                main_subjects.append(chunk)
    if main_subjects:
        main_subject = main_subjects[0]
    else:
        main_subject = None
    # Find the related words (adjectives, verbs) to the main subject
    related_words = defaultdict(list)
    for sentence in sentences:
        doc = nlp(sentence.lower())
        for tok in doc:
            if tok.text in uninformative_words or not tok.text.isalnum():
                continue
            if (tok.pos_ == 'NOUN') and (main_subject is None or (tok.text != main_subject.text)):
                related_words[sentence].append(tok.text)
    # Create image prompts
    image_prompts = []
    for sentence, related in related_words.items():
        if main_subject is not None:
            prompt = f"{main_subject.text} {' '.join(related)} photorealistic"
        else:
            prompt = f"{sentence} photorealistic"
        image_prompts.append(prompt)
    # If we couldn't generate enough prompts, duplicate the existing ones
    if len(image_prompts) < num_prompts:
        print(f"Could only generate {len(image_prompts)} unique prompts out of the requested {num_prompts}. Duplicating prompts...")
        i = 0
        while len(image_prompts) < num_prompts:
            image_prompts.append(image_prompts[i])
            i = (i + 1) % len(image_prompts)  # cycle through existing prompts
    print("\nGenerated Image Prompts:")
    for idx, prompt in enumerate(image_prompts, start=1):
        print(f"{idx}: {prompt}")
    # Ask the user whether they want to proceed or enter their own prompts
    user_input = input("\nDo you want to proceed with these prompts? (y/n): ")
    if user_input.lower() == "y":
        return image_prompts
    elif user_input.lower() == "n":
        user_prompts = []
        print("\nEnter your own image prompts:")
        for i in range(num_prompts):
            user_prompt = input(f"Prompt {i+1}: ")
            user_prompts.append(user_prompt)
        return user_prompts
    else:
        print("Invalid input. Please enter 'y' to proceed with the generated prompts or 'n' to enter your own prompts.")

# Function to generate images using Stable Diffusion XL




In [7]:
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from transformers import AutoTokenizer, pipeline as hf_pipeline
import torch

def generate_story_from_question(question):
    model = "tiiuae/falcon-7b-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model)

    tf_pipeline = hf_pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto",
        max_length=200,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=tf_pipeline, model_kwargs={'temperature': 0})

    template = """

    Question: {question}
    Answer:"""

    prompt = PromptTemplate(template=template, input_variables=["question"])

    llm_chain = LLMChain(prompt=prompt, llm=llm)

    generated_story = llm_chain.run(question)

    return generated_story


In [8]:
import random
import sys
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

def generate_images(image_prompts, num_inference_steps=4, use_lora=False):
    base = "stabilityai/stable-diffusion-xl-base-1.0"
    repo = "ByteDance/SDXL-Lightning"
    device = "cuda"

    unet = UNet2DConditionModel.from_config(
        base,
        subfolder="unet",
        ).to(device, torch.float16)

    unet.load_state_dict(
        load_file(
            hf_hub_download(
                repo,
                f"sdxl_lightning_{num_inference_steps}step_{'lora' if use_lora else 'unet'}.safetensors",
                ),
            device=device,
            ),
        )

    pipe = StableDiffusionXLPipeline.from_pretrained(
        base,
        unet=unet,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16",
        ).to(device)

    if use_lora:
        pipe.load_lora_weights(hf_hub_download(repo, f"sdxl_lightning_{num_inference_steps}step_{'lora' if use_lora else 'unet'}.safetensors"))
        pipe.fuse_lora()

    pipe.scheduler = EulerDiscreteScheduler.from_config(
        pipe.scheduler.config,
        timestep_spacing="trailing",
        )

    generated_images = []

    for prompt in image_prompts:
        seed = random.randint(0, sys.maxsize)
        images = pipe(
            prompt=prompt,
            guidance_scale=0.0,
            num_inference_steps=num_inference_steps,
            generator=torch.Generator("cuda").manual_seed(seed),
        ).images
        generated_images.extend(images)

    return generated_images


In [9]:
from gtts import gTTS
import os

def generate_audio(text, audio_file="output_audio.mp3"):
    # Generate audio using Google Text-to-Speech
    tts = gTTS(text=text, lang='en')
    # Save the audio to a file
    tts.save(audio_file)
    return audio_file


In [10]:
from collections import defaultdict

from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import numpy as np

def generate_video(images, audio_file):
    clips = []
    for image in images:
        # Convert PIL Image to numpy array
        image_array = np.array(image)
        # Create ImageClip from numpy array
        clip = ImageClip(image_array)
        # Set duration for each image clip
        clips.append(clip.set_duration(5))

    # Concatenate image clips into a video
    video = concatenate_videoclips(clips, method="compose")
    # Load audio file
    audio = AudioFileClip(audio_file)
    # Set audio for the video
    video = video.set_audio(audio)
    # Write video file
    video.write_videofile("output.mp4", fps=24)


def main():
    # Generate story plot
    question = input("Ask a question to generate a story: ")

    # Generate story based on the question
    story = generate_story_from_question(question)
    # Extract image prompts from the story
    image_prompts = extract_image_prompts(story)
    # Generate images based on the prompts
    images = generate_images(image_prompts)
    # Generate audio from the story
    audio_file = generate_audio(story)
    # Generate video with subtitles
    generate_video(images, audio_file)

if __name__ == "__main__":
    main()


Ask a question to generate a story: generate a story of a boy name junaid who is playing in a garden with a dog


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4


In [11]:
from google.colab import files

# Download the video file
files.download("output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>